In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [10]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory('train',
                                              target_size=(128, 128),
                                              batch_size=32,
                                              class_mode='binary')

test_data = test_datagen.flow_from_directory('test',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 9605 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_data,
          steps_per_epoch=train_data.samples // 32,
          epochs=20,
          validation_data=test_data,
          validation_steps=test_data.samples // 32)

Epoch 1/10
300/300 [==============================] - 101s 332ms/step - loss: 0.4728 - accuracy: 0.7671 - val_loss: 0.3251 - val_accuracy: 0.8579
Epoch 2/10
300/300 [==============================] - 121s 403ms/step - loss: 0.3512 - accuracy: 0.8520 - val_loss: 0.2692 - val_accuracy: 0.8831
Epoch 3/10
300/300 [==============================] - 157s 523ms/step - loss: 0.3049 - accuracy: 0.8762 - val_loss: 0.2503 - val_accuracy: 0.8901
Epoch 4/10
300/300 [==============================] - 147s 487ms/step - loss: 0.2986 - accuracy: 0.8777 - val_loss: 0.2365 - val_accuracy: 0.8992
Epoch 5/10
300/300 [==============================] - 121s 401ms/step - loss: 0.2827 - accuracy: 0.8839 - val_loss: 0.2487 - val_accuracy: 0.9012
Epoch 6/10
300/300 [==============================] - 116s 388ms/step - loss: 0.2709 - accuracy: 0.8921 - val_loss: 0.2325 - val_accuracy: 0.8992
Epoch 7/10
300/300 [==============================] - 147s 487ms/step - loss: 0.2737 - accuracy: 0.8902 - val_loss: 0.2452 -

In [14]:
loss, accuracy = model.evaluate(test_data)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

32/32 [==============================] - 3s 77ms/step - loss: 0.2365 - accuracy: 0.9000
Test loss: 0.2365
Test accuracy: 0.9000


In [15]:
model.save('weights.h5')

c:\Users\basav\anaconda3\envs\balaji\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
from tensorflow.keras.models import load_model
model = load_model('weights.h5')
def preprocess_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = img / 255.0 
    img = np.expand_dims(img, axis=0) 
    return img
def predict_image(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)

    if prediction > 0.5:
        return 'benign'
    else:
        return 'malignant'

image_path = 'test/malignant/melanoma_10105.jpg'
result = predict_image(image_path)
print(f'The image is a {result}')
image_path


1/1 [==============================] - 0s 109ms/step
The image is a benign


'test/malignant/melanoma_10105.jpg'